In [21]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import nltk
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from nltk.tokenize import word_tokenize
import re
from sklearn.metrics import balanced_accuracy_score, f1_score
from pathlib import Path
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters

In [19]:
stop_words = [word.strip() for word in open('stopwords-pt.txt', mode='r', encoding='utf8')]

In [24]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle')

In [29]:
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]
    
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    porter_stemmer=nltk.PorterStemmer()
    words = [porter_stemmer.stem(word) for word in words]
    return words

def get_total_words(row):
    return len(word_tokenize(row['text']))

def write_predictions(predictions, out_path):
    count = 0

    with open(out_path, mode='w', encoding='utf-8') as out_file:
        print('Saving predictions to %s' % out_path)
        out_file.write('id,category\n')
        idx = 0
        for result in predictions:
            count += 1
            out_file.write(str(idx) + ',' + result + '\n')
            idx += 1
            if count % 100 == 0:
                print('Predicted %d sentences' % count)
    out_file.close()
    print('Finished predicting %d sentences' % count)
    print('Results saved in %s' % Path(out_path).absolute())

In [20]:
stop_words.extend(Tokenizer(' '.join(stop_words)))
stop_words = sorted(stop_words)
stop_words

['a',
 'a',
 'a',
 'acerca',
 'acerca',
 'adeu',
 'adeus',
 'agora',
 'agora',
 'ainda',
 'ainda',
 'al',
 'alem',
 'alem',
 'algma',
 'algmas',
 'algo',
 'algo',
 'alguma',
 'algumas',
 'algun',
 'alguns',
 'ali',
 'ali',
 'além',
 'amba',
 'ambas',
 'ambo',
 'ambos',
 'amo',
 'amo',
 'amo',
 'ano',
 'ano',
 'ano',
 'anos',
 'ant',
 'antes',
 'ao',
 'ao',
 'ao',
 'aond',
 'aonde',
 'aos',
 'ap',
 'apena',
 'apenas',
 'apo',
 'apoio',
 'apoio',
 'apontar',
 'apontar',
 'apos',
 'após',
 'aquel',
 'aquel',
 'aquela',
 'aquela',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aqui',
 'aqui',
 'aquilo',
 'aquilo',
 'as',
 'as',
 'assim',
 'assim',
 'at',
 'atr',
 'atrav',
 'através',
 'atrás',
 'até',
 'aí',
 'baixo',
 'baixo',
 'bastant',
 'bastante',
 'bem',
 'bem',
 'boa',
 'boa',
 'boa',
 'boas',
 'bom',
 'bom',
 'bon',
 'bons',
 'breve',
 'breve',
 'c',
 'cada',
 'cada',
 'caminho',
 'caminho',
 'catorz',
 'catorze',
 'cedo',
 'cedo',
 'cento',
 'cento',
 'certament',
 'certamente',


In [4]:
classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('colext', TextSelector('text')),
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stop_words, ngram_range=(1,2))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
        ])),
        ('words', Pipeline([
            ('wordext', NumberSelector('TotalWords')),
            ('wscaler', StandardScaler()),
        ])),
    ])),
    ('clf', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
    ])

In [5]:
train_df = pd.read_csv('train_text_label.csv')

In [6]:
dev_df = pd.read_csv('dev_text.csv')

In [9]:
X = train_df[['text']]
Y = train_df['label']

In [10]:
classifier.fit(X, Y)
preds = classifier.predict(X)

/home/pedro/anaconda3/envs/allennlp/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['adeu', 'al', 'algma', 'alguma', 'algun', 'amba', 'ambo', 'amo', 'ant', 'aond', 'ap', 'apena', 'apo', 'aquel', 'at', 'atr', 'atrav', 'bastant', 'bon', 'c', 'catorz', 'certament', 'corrent', 'd', 'daquel', 'demai', 'depoi', 'desd', 'dess', 'dest', 'dever', 'dezanov', 'dezassei', 'dezasset', 'diant', 'dispo', 'diverso', 'doi', 'dua', 'durant', 'ent', 'entr', 'ess', 'est', 'estamo', 'estejamo', 'estev', 'estiv', 'estivemo', 'estivermo', 'estivess', 'estivest', 'f', 'fa', 'far', 'faze', 'fazei', 'fazemo', 'fomo', 'formo', 'foss', 'fost', 'grand', 'h', 'hajamo', 'havemo', 'houv', 'houvemo', 'houveremo', 'houvermo', 'houvess', 'is', 'ist', 'j', 'l', 'long', 'ltimo', 'm', 'ma', 'mai', 'mediant', 'meno', 'mero', 'mese', 'n', 'naquel', 'ness', 'nest', 'nhamo', 'noit', 'nun', '

In [11]:
balanced_accuracy_score(Y, preds)

0.9996073812328229

In [12]:
f1_score(Y, preds, average='micro')

0.9996199645300228

In [13]:
write_predictions(classifier.predict(dev_df[['text']]), 'submission_xgboost_words.csv')

Saving predictions to submission_xgboost_words.csv
Predicted 100 sentences
Predicted 200 sentences
Predicted 300 sentences
Predicted 400 sentences
Predicted 500 sentences
Predicted 600 sentences
Predicted 700 sentences
Predicted 800 sentences
Predicted 900 sentences
Predicted 1000 sentences
Predicted 1100 sentences
Predicted 1200 sentences
Predicted 1300 sentences
Predicted 1400 sentences
Predicted 1500 sentences
Predicted 1600 sentences
Predicted 1700 sentences
Predicted 1800 sentences
Predicted 1900 sentences
Predicted 2000 sentences
Predicted 2100 sentences
Predicted 2200 sentences
Predicted 2300 sentences
Predicted 2400 sentences
Predicted 2500 sentences
Predicted 2600 sentences
Predicted 2700 sentences
Predicted 2800 sentences
Predicted 2900 sentences
Predicted 3000 sentences
Predicted 3100 sentences
Predicted 3200 sentences
Predicted 3300 sentences
Predicted 3400 sentences
Predicted 3500 sentences
Predicted 3600 sentences
Predicted 3700 sentences
Predicted 3800 sentences
Predicte

In [25]:
texts, labels = [], []
for idx, row in train_df.iterrows():
    text = row['text']
    sentences = sentence_tokenizer.tokenize(text)
    for sentence in sentences:
        texts.append(sentence)
        labels.append(row['label'])

In [26]:
train_extended_df = pd.DataFrame({'text': texts, 'label': labels})
train_extended_df

,text,label
0,Casa da Barra Funda tem clima roceiro e receit...,comida
1,"No Quem Quer Pão 75, casa de clima roceiro em ...",comida
2,"O café coado é de um lote limitado, de grãos d...",comida
3,"Se é final da tarde, um pão de queijo robusto ...",comida
4,"Se é para encerrar o almoço, vai bem o brigade...",comida
5,É tentador também sugerir uma fatia dos bolos ...,comida
6,"Meu palpite, porém, é que ainda falta para que...",comida
7,A pequena casa é embalada por música popular b...,comida
8,"O tom hospitaleiro avança para a cozinha, orqu...",comida
9,"Suas receitas são caseiras e, no balanço, sabo...",comida


In [28]:
train_extended_df['TotalWords'] = train_extended_df.apply(lambda row: get_total_words(row), axis=1)
train_extended_df

,text,label,TotalWords
0,Casa da Barra Funda tem clima roceiro e receit...,comida,11
1,"No Quem Quer Pão 75, casa de clima roceiro em ...",comida,25
2,"O café coado é de um lote limitado, de grãos d...",comida,27
3,"Se é final da tarde, um pão de queijo robusto ...",comida,38
4,"Se é para encerrar o almoço, vai bem o brigade...",comida,33
5,É tentador também sugerir uma fatia dos bolos ...,comida,40
6,"Meu palpite, porém, é que ainda falta para que...",comida,24
7,A pequena casa é embalada por música popular b...,comida,43
8,"O tom hospitaleiro avança para a cozinha, orqu...",comida,31
9,"Suas receitas são caseiras e, no balanço, sabo...",comida,11


In [31]:
X = train_extended_df[['text', 'TotalWords']]
Y = train_extended_df['label']

In [32]:
classifier.fit(X, Y)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('colext', TextSelector(field='text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='co...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))])